<a href="https://colab.research.google.com/github/prasadt1/Best-websites-a-programmer-should-visit/blob/master/bytebyteaicohort%20/lm_playground_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Project 1: Build an LLM Playground

Welcome! In this project, you’ll learn foundations of large language models (LLMs). We’ll keep the code minimal and the explanations high‑level so that anyone who can run a Python cell can follow along.  

We'll be using Google Colab for this project. Colab is a free, browser-based platform that lets you run Python code and machine learning models without installing anything on your local computer. Click the button below to open this notebook directly in Google Colab and get started!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-eng-projects/blob/main/project_1/lm_playground.ipynb)

---
## Learning Objectives  
* **Tokenization** and how raw text is tokenized into a sequene of discrete tokens
* Inspect **GPT2** and **Transformer architecture**
* Loading pre-trained LLMs using **Hugging Face**
* **Decoding strategies** to generate text from LLMs
* Completion versus **intrusction fine-tuned** LLMs


Let's get started!

In [ ]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)

torch 2.8.0+cu126 | transformers 4.57.0


# 1 - Tokenization

A neural network can’t digest raw text. They need **numbers**. Tokenization is the process of converting text into IDs. In this section, you'll learn how tokenization is implemented in practice.

Tokenization methods generally fall into three categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word‑level tokenization

Split text on whitespace and store each **word** as a token.

In [ ]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
#The [PAD] token is used for padding sequences to equal length,
#and [UNK] represents unknown words not in your vocabulary
vocab = []
PAD, UNK = "[PAD]", "[UNK]"
word2id = {}
id2word = {}
set_from_corpus = set()

#my code begins
for sentence in corpus:
    set_from_corpus.update(sentence.lower().split(" "))

vocab = list(set_from_corpus)
vocab.insert(0,PAD)
vocab.insert(1,UNK)


for index, word in enumerate(vocab):
    word2id[word] = index

for index, word in enumerate(vocab):
    id2word[index] = word


#my code ends

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])

# 3. Encode / decode
def encode(text):

    #my code begins
    words = text.split( " ")

    # Convert each word to its ID (get returns 1 if word not found)
    ids = [word2id.get(word, 1) for word in words]

    return ids

    print(ids)
#my code ends

def decode(ids):
    #my code begins
   # Convert each ID to its word (return "[UNK]" if ID not found)
    words = [id2word.get(id, "[UNK]") for id in ids]

    # Join the words with spaces to form a sentence
    decoded_text = " ".join(words)

    return decoded_text

    print(decoded_text)
#my code ends

# 4. Demo
sample = "The brown horse jumps"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)

Vocabulary size: 21 words
First 15 vocab entries: ['[PAD]', '[UNK]', 'to', 'next', 'language', 'tokenization', 'quick', 'dog', 'fox', 'numbers', 'converts', 'the', 'lazy', 'text', 'over']

Input text : The brown horse jumps
Token IDs  : [1, 18, 1, 20]
Decoded    : [UNK] brown [UNK] jumps


Word-level tokenization has two major limitations:
1. Large vocabulary size
2. Out-of-vocabulary (OOV) issue

### 1.2 - Character‑level tokenization

Every single character (including spaces and emojis) gets its own ID. This guarantees zero out‑of‑vocabulary issues but very long sequences.

In [ ]:
# 1. Build a fixed vocabulary # a–z + A–Z + padding + unkwown
import string

vocab = []
char2id = {}
id2char = {}
"""
YOUR CODE HERE
"""

#my code starts here
PAD, UNK = "[PAD]", "[UNK]"
vocab = [PAD, UNK] + list(string.ascii_letters)

for index, word in enumerate(vocab):
    char2id[word] = index

for index, word in enumerate(vocab):
    id2char[index] = word
#my code ends here
print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")

# 2. Encode / decode
def encode(text):
    """
    YOUR CODE HERE
    """
#my code starts here
     # Convert text to list of character IDs
    ids = [char2id.get(char, 1) for char in text]
    return ids
#my code ends here


def decode(ids):
    """
    YOUR CODE HERE
    """
#my code starts here
    chars = [id2char.get(id, "[UNK]") for id in ids]
    decoded_text = "".join(chars)  # Join with NO SPACES
    return decoded_text
#my code ends here

# 3. Demo
sample = "Hello prasad how are you doing?"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 54 (52 letters + 2 specials)

Input text : Hello prasad how are you doing?
Token IDs  : [35, 6, 13, 13, 16, 1, 17, 19, 2, 20, 2, 5, 1, 9, 16, 24, 1, 2, 19, 6, 1, 26, 16, 22, 1, 5, 16, 10, 15, 8, 1]
Decoded    : Hello[UNK]prasad[UNK]how[UNK]are[UNK]you[UNK]doing[UNK]


### 1.3 - Subword‑level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** the most common character and gorup them into new tokens. For example, the word `unbelievable` might turn into three tokens: `["un", "believ", "able"]`. This approach strikes a balance between word-level and character-level methods and fix their limitations.

For example, `BPE` algorithm forms the vocabulary using the following steps:
1. **Start with bytes** → every character is its own token.  
2. **Count all adjacent pairs** in a huge corpus.  
3. **Merge the most frequent pair** into a new token.  
   *Repeat steps 2-3* until you hit the target vocab size (e.g., 50 k).

Let's see `BPE` in practice.

In [ ]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE).
# Refer to  https://huggingface.co/docs/transformers/en/fast_tokenizers

from transformers import AutoTokenizer
#my code
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE)
bpe_tok = AutoTokenizer.from_pretrained("gpt2")
#my code ends

# 2. Encode / decode
def encode(text):
    """
    YOUR CODE HERE
    """
    #my code starts here
    #convert_tiktoken_to_fast(encoding, "config/save/dir")

    return bpe_tok.encode(text)
    #my code ends here

def decode(ids):
    """
    YOUR CODE HERE
    """
    #my code starts here
    return bpe_tok.decode(ids)
    #my code ends here

# 3. Demo
sample = "Unbelievable tokenization powers Prasad! 🚀"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Tokens     :", bpe_tok.convert_ids_to_tokens(ids))
print("Decoded    :", recovered)


Input text : Unbelievable tokenization powers Prasad! 🚀
Token IDs  : [3118, 6667, 11203, 540, 11241, 1634, 5635, 1736, 292, 324, 0, 12520, 248, 222]
Tokens     : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', 'ĠPr', 'as', 'ad', '!', 'ĠðŁ', 'ļ', 'Ģ']
Decoded    : Unbelievable tokenization powers Prasad! 🚀


### 1.4 - TikToken

`tiktoken` is a production-ready library which offers high‑speed tokenization used by OpenAI models.  
Let's compare the older **gpt2** encoding with the newer **cl100k_base** used in GPT‑4.

In [ ]:
# Use gpt2 and cl100k_base to encode and decode the following text
# Refer to https://github.com/openai/tiktoken
import tiktoken

encodings = [
    ("gpt2", tiktoken.get_encoding("gpt2")),
    ("cl100k_base", tiktoken.get_encoding("cl100k_base")),
]

sentence = "The 🌟 star-player scored 40 points!"

for name, enc in encodings:
    print(f"\n=== {name} ===")
    print("Vocabulary size:", enc.n_vocab)

    # Encode the sample sentence
    ids = enc.encode(sentence)
    tokens = [enc.decode([i]) for i in ids]
    print(f"Sentence splits into {len(ids)} tokens:")
    print(list(zip(tokens, ids)))

    # Show a few arbitrary token→ID examples from the vocab
    some_ids = [0, 1, 2, 198, 50256]
    print("Sample tokens from the vocabulary:")
    print([(enc.decode([i]), i) for i in some_ids])


=== gpt2 ===
Vocabulary size: 50257
Sentence splits into 11 tokens:
[('The', 464), (' �', 12520), ('�', 234), ('�', 253), (' star', 3491), ('-', 12), ('player', 7829), (' scored', 7781), (' 40', 2319), (' points', 2173), ('!', 0)]
Sample tokens from the vocabulary:
[('!', 0), ('"', 1), ('#', 2), ('\n', 198), ('<|endoftext|>', 50256)]

=== cl100k_base ===
Vocabulary size: 100277
Sentence splits into 11 tokens:
[('The', 791), (' �', 11410), ('�', 234), ('�', 253), (' star', 6917), ('-player', 43467), (' scored', 16957), (' ', 220), ('40', 1272), (' points', 3585), ('!', 0)]
Sample tokens from the vocabulary:
[('!', 0), ('"', 1), ('#', 2), ('\n', 198), ('parable', 50256)]


Experiment: try new sentences, emojis, code snippets, or other languages. If you are interested, try implementing the BPE algorithm yourself.

### 1.5 - Key Takeaways

* **Word‑level**: simple but brittle (OOV problems).  
* **Character‑level**: robust but produces long sequences.  
* **BPE / Byte‑Level BPE**: middle ground used by most LLMs.  
* **tiktoken**: shows how production models tokenize with pre‑trained sub‑word vocabularies.

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer applies a simple operation (matrix multiplication, attention, etc.). Stacking hundreds of these layers lets the model capture patterns and statistical relations from text. The final output is a vector of scores that says, “how likely is each possible token to come next?”

> Think of the whole network as **one gigantic equation** whose parameters were tuned during training to minimize prediction error.



### 2.1 - A Single `Linear` Layer

Before we explore Transformer, let’s start tiny:

* A **Linear layer** performs `y = Wx + b`  
  * `x` – input vector  
  * `W` – weight matrix (learned)  
  * `b` – bias vector (learned)

Although this looks basic, chaining thousands of such linear transforms (with nonlinearities in between) gives neural nets their expressive power.


Purpose	To define and initialize the layer's learnable parameters: the weights (W) and the biases (b).
Use torch.nn.Parameter or torch.nn.Linear (if the goal is a simple wrapper) to create these tensors.
**Inputs	in_features**: The size of the input vector x (the number of input units). **out_features**: The size of the output vector y (the number of output units).
The weights matrix W will have a shape of (out_features, in_features) and the bias vector b will have a shape of (out_features). **bold text**

In [ ]:
import torch.nn as nn
class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        return torch.matmul(x, self.weight.t()) + self.bias

lin = Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))

Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[-0.5613,  1.4410,  0.6801],
        [-0.1885,  0.0975,  1.2891]], requires_grad=True)
Bias   : Parameter containing:
tensor([ 0.5932, -1.6240], requires_grad=True)
Output : tensor([-1.0690, -1.2654], grad_fn=<AddBackward0>)


In [ ]:
import torch.nn as nn, torch

lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[-0.5121, -0.0291,  0.3108],
        [ 0.3442, -0.5261,  0.0264]], requires_grad=True)
Bias   : Parameter containing:
tensor([0.5342, 0.1090], requires_grad=True)
Output : tensor([0.2066, 0.9926], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

Below, we load the smallest public GPT-2 (124 M parameters), grab its *first* block, and inspect the pieces.


In [3]:
import torch
from transformers import GPT2LMHeadModel

# Load the 124 M-parameter GPT-2 and inspect its layers (12 layers)
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
print(gpt2)
block = gpt2.transformer.h[0] # GPT-2 has 12 such layers
for name, module in block.named_children():
    print(f"{name:7s} → {module.__class__.__name__}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
ln_1    → LayerNorm
attn    → GPT2Attention
ln_2    →

In [4]:
# Run a tiny forward pass through the first block
seq_len = 8  # Define the fixed length for our input sequence (8 tokens)
dummy_tokens = torch.randint(0, gpt2.config.vocab_size, (1, seq_len)) # Create a dummy input tensor: 1 batch, 8 random tokens (IDs)
                                                                     # The token IDs are randomly sampled from the model's vocabulary

with torch.no_grad(): # Disable gradient computation, which is essential for inference/testing to save memory
    # Embed tokens + positions the same way GPT-2 does
    hidden = (
        gpt2.transformer.wte(dummy_tokens) +  # Get the Word Token Embeddings (wte) for the dummy tokens
        gpt2.transformer.wpe(torch.arange(seq_len)) # Get the Positional Embeddings (wpe) for positions 0 to 7
    )
    # The two embeddings (wte and wpe) are summed to get the initial input vector for the first block

    # Forward through one layer
    out = block(hidden, layer_past=None, use_cache=False)[0] # Pass the combined embedding through the specific transformer 'block'
                                                              # layer_past=None and use_cache=False means it's a standard non-autoregressive pass
                                                              # [0] is used to extract the main output tensor from the block's return tuple

print("\nOutput shape :", out.shape) # Print the shape of the resulting tensor: (batch_size, seq_len, hidden_size)


Output shape : torch.Size([1, 8, 768])


### 2.3 - Inside GPT-2

GPT-2 is just many of those modules arranged in a repeating *block*. Let's print the modules inside the Transformer.

In [5]:
# Print the name and modules inside gpt2
for name, module in gpt2.transformer.named_children():
    print(f"{name:7s} → {module.__class__.__name__}")

wte     → Embedding
wpe     → Embedding
drop    → Dropout
h       → ModuleList
ln_f    → LayerNorm


As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

Passing a token sequence through an **LLM** yields a tensor of **logits** with shape  
`(batch_size, seq_len, vocab_size)`.  
Applying `softmax` on the last dimension turns those logits into probabilities.

The cell below feeds an 8-token dummy sequence, prints the logits shape, and shows the five most likely next tokens for the final position.


In [2]:
import torch, torch.nn.functional as F  # Import PyTorch and the functional module (for softmax)
from transformers import GPT2LMHeadModel, GPT2TokenizerFast # Import necessary classes from the Hugging Face library

# Load gpt2 model and tokenizer

try:
    gpt2 # Try to reference the model object
except NameError: # If 'gpt2' hasn't been defined yet (first run)
    gpt2 = GPT2LMHeadModel.from_pretrained("gpt2") # Load the pre-trained GPT-2 model with a language modeling head
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2") # Load the fast tokenizer corresponding to the GPT-2 model

# Tokenize input text
text = "Hello my name" # The input text prompt
input_ids = tokenizer(text, return_tensors="pt").input_ids # Convert the text into a tensor of token IDs (shape: 1, seq_len)


# Get logits by passing the ids to the gpt2 model.
with torch.no_grad(): # Disable gradient calculation, as this is an inference (prediction) task
    logits = gpt2(input_ids).logits # Perform a full forward pass through the GPT-2 model
                                   # The .logits attribute extracts the prediction scores (logits)
                                   # Logits shape: (batch_size, seq_len, vocab_size)

print("Logits shape :", logits.shape) # Output the shape of the raw prediction scores


# Predict next token
probs = F.softmax(logits[0, -1], dim=-1) # Focus on the logits for the *last* token's prediction (index -1)
                                         # [0, -1] selects the last row of logits for the first batch item
                                         # F.softmax converts the raw logits into probability distribution (sum = 1)
topk = torch.topk(probs, 5) # Find the top 5 highest probabilities and their corresponding indices (token IDs)

print("\nTop-5 predictions for the next token:")
for idx, p in zip(topk.indices.tolist(), topk.values.tolist()): # Iterate through the top-5 token IDs (indices) and probabilities
    print(f"{tokenizer.decode([idx]):>10s}  —  {p:.4f}") # Decode the token ID back into a human-readable word/piece and print the probability

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Logits shape : torch.Size([1, 3, 50257])

Top-5 predictions for the next token:
        is  —  0.7773
         ,  —  0.0373
        's  —  0.0332
       was  —  0.0127
       and  —  0.0076


### 2.5 - Key Takeaway

A language model is nothing mystical: it’s a *huge composition* of small, understandable layers trained to predict the next token in a sequence of tokens.

# 3 - Generation
Once an LLM is trained to predict the probabilities, we can generate text from the model. This process is called decoding or sampling.

At each step, the LLM outputs a **probability distribution** over the next token. It is the job of the decoding algorithm to pick the next token, and move on to the next token. There are different decoding algorithms and hyper-parameters to control the generaiton:
* **Greedy** → pick the single highest‑probability token each step (safe but repetitive).  
* **Top‑k / Nucleus (top‑p)** → sample from a subset of likely tokens (adds variety).
* **beam** -> applies beam search to pick tokens
* **Temperature** → a *creativity* knob. Higher values flatten the probability distribution.

### 3.1 - Greedy decoding

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
}
tokenizers, models = {}, {}
# Load models and tokenizers
device = "cuda" if torch.cuda.is_available() else "cpu"
for key, mid in MODELS.items():
    tok = AutoTokenizer.from_pretrained(mid)
    mdl = AutoModelForCausalLM.from_pretrained(mid).eval().to(device)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    mdl.config.pad_token_id = tok.pad_token_id
    tokenizers[key], models[key] = tok, mdl
    print(f"Loaded {mid} as {key}")

def generate(model_key, prompt, strategy="greedy", max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]
    enc = tok(prompt, return_tensors="pt").to(mdl.device)
    gen_args = dict(**enc, max_new_tokens=max_new_tokens, pad_token_id=tok.pad_token_id)
    if strategy=="greedy":
        gen_args["do_sample"]=False
    elif strategy=="top_k":
        gen_args.update(dict(do_sample=True, top_k=50, temperature=0.2))
    elif strategy=="top_p":
        gen_args.update(dict(do_sample=True, top_p=0.9, temperature=0.2))
    out = mdl.generate(**gen_args)
    return tok.decode(out[0], skip_special_tokens=True)


Loaded gpt2 as gpt2


In [4]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate("gpt2", prompt, "greedy", 80))



== GPT-2 | Greedy ==
Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== GPT-2 | Greedy ==
What is 2+2?

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

== GPT-2 | Greedy ==
Suggest a party theme.

The party theme is a simple, simple, and fun way to get your friends to join you.

The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple


Naively picking the single best token every time has the following issues in practice:

* **Loop**: “The cat is is is…”  
* **Miss long-term payoff**: the highest-probability word *now* might paint you into a boring corner later.

### 3.2 - Top-k or top-p sampling

In [8]:

tests=["How are you","What is India?", "Suggest a movie."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate("gpt2", prompt, "top-p", 40))



== GPT-2 | Top-p ==
How are you doing?

I'm doing a lot of work on my website. I'm doing a lot of work on my website. I'm doing a lot of work on my website. I'm doing

== GPT-2 | Top-p ==
What is India?

India is a country of about 1.5 billion people. It is the second largest economy in the world after China. It is the second largest economy in the world after the United States.

== GPT-2 | Top-p ==
Suggest a movie.

The movie is called "The Last of Us" and it's about a group of people who are trying to save the world from a zombie apocalypse. It's about a group of people who


### 3.3 - Try It Yourself

1. Scroll to the list called `tests`.
2. Swap in your own prompts or tweak the decoding strategy.  
3. Re‑run the cell and compare the vibes.

> **Tip:** Try the same prompt with `greedy` vs. `top_p` (0.9) and see how the tone changes. Notice especially how small temperature tweaks can soften or sharpen the prose.

* `strategy`: `"greedy"`, `"beam"`, `"top_k"`, `"top_p"`  
* `temperature`: `0.2 – 2.0`  
* `k` or `p` thresholds



# 4 - Completion vs. Instruction-tuned LLMs

We have seen that we can use GPT2 model to pass an input text and generate a new text. However, this model only continues the provided text. It is not engaging in a dialouge-like conversation and cannot be helpful by answering instructions. On the other hand, **instruction-tuned LLMs** like `Qwen-Chat` go through an extra training stage called **post-training** after the base “completion” model is finished. Because of post-training step, an instruction-tuned LLM will:

* **Read the entire prompt as a request,** not just as text to mimic.  
* **Stay in dialogue mode**. Answer questions, follow steps, ask clarifying queries.  
* **Refuse or safe-complete** when instructions are unsafe or disallowed.  
* **Adopt a consistent persona** (e.g., “Assistant”) rather than drifting into story continuation.


### 4.1 - Qwen1.5-8B vs. GPT2

In the code below we’ll feed the same prompt to:

* **GPT-2 (completion-only)** – it will simply keep writing in the same style.  
* **Qwen-Chat (instruction-tuned)** – it will obey the instruction and respond directly.

Comparing the two outputs makes the difference easy to see.

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}
tokenizers, models = {}, {}
# Load models and tokenizers
tokenizers, models = {}, {}
# Load models and tokenizers
device = "cuda" if torch.cuda.is_available() else "cpu"
for key, mid in MODELS.items():
    tok = AutoTokenizer.from_pretrained(mid)
    mdl = AutoModelForCausalLM.from_pretrained(mid).eval().to(device)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    mdl.config.pad_token_id = tok.pad_token_id
    tokenizers[key], models[key] = tok, mdl
    print(f"Loaded {mid} as {key}")



Loaded gpt2 as gpt2


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

Loaded Qwen/Qwen1.5-1.8B-Chat as qwen



We downloaded two tiny checkpoints: `GPT‑2` (124 M parameters) and `Qwen‑1.5‑Chat` (1.8 B). If the cell took a while, that was mostly network time. Models are stored locally after the first run.

Let's now generate text and compare two models.


In [11]:

tests=[("Once upon a time","greedy"),("What is 2+2?","top_k"),("Suggest a party theme.","top_p")]
for prompt,strategy in tests:
    for key in ["gpt2","qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(key,prompt,strategy,80))



== GPT2 | greedy ==


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== QWEN | greedy ==
Once upon a time, there was a young girl named Lily who lived in a small village nestled among the rolling hills of her home. Lily had always been fascinated by nature and spent most of her days exploring the surrounding forests, streams, and meadows. She loved to sing songs about the birds that sang in the trees, the flowers that bloomed in the fields, and the animals that roamed free.
One day

== GPT2 | top_k ==
What is 2+2?

2+2 is the number of times a character can be cast as a 2+2 character.

2+2 is the number of times a character can be cast as a 2+2 character. 2+2 is the number of times a character ca

# 5. (Optional) A Small LLM Playground

### 5.1 ‑ Interactive Playground

Enter a prompt, pick a model and decoding strategy, adjust the temperature, and press **Generate** to watch the model respond.


In [16]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Make sure models and tokenizers are loaded
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")
#print(f"Tokenizers: {tokenizers}, Models: {models}")
def generate_playground(model_key, prompt, strategy="greedy", temperature=1.0, max_new_tokens=100):
    # Generation code
    tok, mdl = tokenizers[model_key], models[model_key]
    enc = tok(prompt, return_tensors="pt").to(mdl.device)
    gen_args = dict(**enc, max_new_tokens=max_new_tokens, pad_token_id=tok.pad_token_id)
    if strategy == "greedy":
        gen_args["do_sample"] = False
    elif strategy == "top_k":
        gen_args.update(dict(do_sample=True, top_k=50, temperature=temperature))
    elif strategy == "top_p":
        gen_args.update(dict(do_sample=True, top_p=0.9, temperature=temperature))
    else:
        raise ValueError("Unknown strategy")
    out = mdl.generate(**gen_args)
    return tok.decode(out[0], skip_special_tokens=True)

# Your code to build boxes, dropdowns, and other elements in the UI using widgets and creating the UI using widgets.vbox and display.
# Refer to https://ipywidgets.readthedocs.io/en/stable/
prompt_box = widgets.Textarea(
    value="Tell me a fun fact about space.",
    placeholder="Type your prompt here",
    description="Prompt:",
    layout=widgets.Layout(width="100%", height="120px")
)

model_dropdown = widgets.Dropdown(
    options=[("GPT‑2", "gpt2"), ("Qwen‑1.5‑Chat", "qwen")],
    value="gpt2",
    description="Model:"
)

strategy_dropdown = widgets.Dropdown(
    options=[("Greedy", "greedy"), ("Top‑k", "top_k"), ("Top‑p", "top_p")],
    value="greedy",
    description="Strategy:"
)

temperature_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description="Temp:"
)

generate_button = widgets.Button(description="Generate", button_style="primary")
output_area = widgets.Output()

def on_generate(_):
    output_area.clear_output()
    with output_area:
        try:
            result = generate_playground(
                model_dropdown.value,
                prompt_box.value,
                strategy_dropdown.value,
                temperature_slider.value
            )
            display(Markdown(f"**Output:**\n\n{result}"))
        except Exception as e:
            print("Error:", e)

generate_button.on_click(on_generate)

ui = widgets.VBox([
    prompt_box,
    widgets.HBox([model_dropdown, strategy_dropdown, temperature_slider]),
    generate_button,
    output_area
])

display(ui)



## 🎉 Congratulations!

You’ve just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen1.5-8B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
